In [ ]:
pip install "openai>=1.76.0" "pydantic>=2.10,<3" "griffe>=1.5.6,<2" "typing-extensions>=4.12.2,<5" "requests>=2.0,<3" "types-requests>=2.0,<3" "beautifulsoup4" "reportlab" "mcp>=1.6.0,<2; python_version >= '3.10'"

In [ ]:
pip install openai-agents

In [2]:
from __future__ import annotations

import asyncio

from pydantic import BaseModel

from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

In [3]:
class MathHomeworkOutput(BaseModel):
    reasoning: str
    is_math_homework: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)

In [4]:
@input_guardrail
async def math_guardrail(context: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
    """This is an input guardrail function, which happens to call an agent to check if the input
    is a math homework question.
    """
    result = await Runner.run(guardrail_agent, input, context=context.context)
    final_output = result.final_output_as(MathHomeworkOutput)

    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=final_output.is_math_homework,
    )

In [ ]:
agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
)

input_data: list[TResponseInputItem] = []

while True:
    user_input = input("Enter a message: ")
    input_data.append(
        {
            "role": "user",
            "content": user_input,
        }
    )

    try:
        result = await Runner.run(agent, input_data)
        print(result.final_output)
        input_data = result.to_input_list()
    except InputGuardrailTripwireTriggered:
        message = "Sorry, I can't help you with your math homework."
        print(message)
        input_data.append(
            {
                "role": "assistant",
                "content": message,
            }
        )

Enter a message:  Who is the president of Poland?


The current president of Poland is Andrzej Duda.


Enter a message:  Solve this: 2x + 10 = 12


Sorry, I can't help you with your math homework.


Enter a message:  Ile dni rządził król Stefan Batory?


Król Stefan Batory rządził Polską od 1 maja 1576 roku do 12 grudnia 1586 roku, co daje łącznie 3850 dni.


Enter a message:  Podaj różnicę pomiędzy datą śmierci, a datą urodzin Stefana Batorego


Sorry, I can't help you with your math homework.
